# Excel bestanden lezen
## Een aparte engine
Python kan standaard geen Excelbestanden lezen, maar er zijn modules om daar bij te helpen. We moeten dan rekening houden met het feit dat er verschillende bestandsformaten zijn:
- .xls: het oude excelformaat kan gelezen worden met *xlrd*
- .xlsx: het nieuwe excelformaat kan gelezen woren met *openpyxl*
- .ods: het formaat van de gratis concurrent *libreoffice* kan gelezen worden met *odf*

We zullen een .xlsx bestand lezen. Hiervoor moeten we *openpyxl* installeren:
```
pip install openpyxl
```

## Speel je lotto?
We kunnen de statistieken van de lotto-trekkingen downloaden. Omdat men ervan uitgaat dat het bestand gedownload wordt door een browser, moeten we bij request.get() een header meegeven. Wat we daar invullen, maakt niet uit. Dus 'Syntra browser' werkt. 

In [ ]:
from pathlib import Path
import requests
STATISTIEKEN = 'statistieken-lotto-12-25.xlsx'
statistieken_path = Path(STATISTIEKEN)
if not statistieken_path.exists():
    URL='https://www.nationale-loterij.be/content/dam/opp/draw-games/lotto/brand-assets/documents/nl/statistieken-lotto-12-25.xlsx'
    headers = {'User-Agent': 'Syntra browser'}     
    data = requests.get(URL, headers=headers)

    STATISTIEKEN = 'statistieken-lotto-12-25.xlsx'
    with open(STATISTIEKEN, mode='wb') as f:
        f.write(data.content)
else:
    print('Bestand moet niet gedownload worden.')

## Een Excel workbook bestaat uit worksheets
Het bestand bevat verschillende worksheets.

In [ ]:
import openpyxl
excel = openpyxl.load_workbook(STATISTIEKEN)
excel.sheetnames

## Lezen met read_excel()
Omdat er verschillende worksheets zijn, moeten we een sheet_name meegeven. Een table in Excel kan eender waar staan. Daarom geven we ook mee op welke rij de tabel begint (skiprows) en hoeveel rijen de tabel bevat (nrows)

In [ ]:
import pandas as pd

df = pd.read_excel(STATISTIEKEN, sheet_name='Resultaten', engine='openpyxl', skiprows=4, nrows=1488, index_col='Trekkingsdatum')
df.info()

## Zijn de getallen gelijk verdeeld?
Wanneer er 45 getallen getrokken kunnen worden, zou het gemiddelde van alle waarden 22,5 moeten zijn.

In [ ]:
getallen = df.to_numpy()
getallen.mean()

## Een andere manier
We kunnen numpy.unique gebruiken om te tellen hoe dikwijls elk getal voorkomt. 

In [ ]:
import numpy as np
getal, aantallen = np.unique(getallen, return_counts=True)
print(aantallen)
df_getallen = pd.DataFrame({'aantallen':aantallen}, index=getal)
df_getallen.info()


## Een barplot met de verdeling van de 45 lottogetallen
Dat is de beste plot om te zien of de waarden gelijkmatig verdeeld zijn.

In [ ]:
df_getallen.plot(kind='bar')

## Plotten van de gegevens met matplotlib
We kunnen het ook met Matplotlib doen.

In [ ]:
import matplotlib.pyplot as plt
plt.bar(df_getallen.index, df_getallen['aantallen'])
plt.show()